In [1]:
import numpy as np 
from sklearn import datasets 
from systemml.mllearn import LinearRegression 
# Load the diabetes dataset 
diabetes = datasets.load_diabetes() 
# Use only one feature 
diabetes_X = diabetes.data[:, np.newaxis, 2] 
# Split the data into training/testing sets 
X_train = diabetes_X[:-20] 
X_test = diabetes_X[-20:] 
# Split the targets into training/testing sets 
y_train = diabetes.target[:-20] 
y_test = diabetes.target[-20:] # Create linear regression object 
regr = LinearRegression(spark, fit_intercept=True, C=float("inf"), solver='direct-solve') 
# Train the model using the training sets 
regr.fit(X_train, y_train) 
y_predicted = regr.predict(X_test) 
print('Residual sum of squares: %.2f' % np.mean((y_predicted - y_test) ** 2))

/root/spark/python/pyspark/sql/session.py:714: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 0.8.0 must be installed; however, it was not found.
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)



Welcome to Apache SystemML!
Version 1.3.0-SNAPSHOT
BEGIN LINEAR REGRESSION SCRIPT
Reading X and Y...
Calling the Direct Solver...
Computing the statistics...
AVG_TOT_Y,153.36255924170615
STDEV_TOT_Y,77.21853383600028
AVG_RES_Y,1.3705729547600037E-14
STDEV_RES_Y,63.03850633759284
DISPERSION,3973.853281274733
R2,0.33513125068675453
ADJUSTED_R2,0.3335482298550564
R2_NOBIAS,0.33513125068675453
ADJUSTED_R2_NOBIAS,0.3335482298550564
Writing the output matrix...
END LINEAR REGRESSION SCRIPT
SystemML Statistics:
Total execution time:		0.062 sec.
Number of executed Spark inst:	2.


[Stage 0:>                                                       (0 + 15) / 163]
[Stage 0:=======================>                               (71 + 15) / 163]
SystemML Statistics:
Total execution time:		0.002 sec.
Number of executed Spark inst:	1.

                                                                                
                                                                                

Resi

In [1]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Input, Dense, Conv1D, Conv2D, MaxPooling2D, Dropout,Flatten
from keras import backend as K
from keras.models import Model
import numpy as np
import matplotlib.pyplot as plt

K.set_image_data_format("channels_first")

from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()


# Expect to see a numpy n-dimentional array of (60000, 28, 28)

type(X_train), X_train.shape, type(X_train)


#This time however, we flatten each of our 28 X 28 images to a vector of 1, 784

X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

# expect to see a numpy n-dimentional array of : (60000, 784) for Traning Data shape and (10000, 784) for Test Data shape
type(X_train), X_train.shape, X_test.shape


#We also use sklearn's MinMaxScaler for normalizing

from sklearn.preprocessing import MinMaxScaler
def scaleData(data):
    # normalize features
    scaler = MinMaxScaler(feature_range=(0, 1))
    return scaler.fit_transform(data)

X_train = scaleData(X_train)
X_test = scaleData(X_test)


# We define the same Keras model as earlier

input_shape = (1,28,28) if K.image_data_format() == 'channels_first' else (28,28, 1)
keras_model = Sequential()
keras_model.add(Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=input_shape, padding='same'))
keras_model.add(MaxPooling2D(pool_size=(2, 2)))
keras_model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
keras_model.add(MaxPooling2D(pool_size=(2, 2)))
keras_model.add(Flatten())
keras_model.add(Dense(512, activation='relu'))
keras_model.add(Dropout(0.5))
keras_model.add(Dense(10, activation='softmax'))
keras_model.summary()


# Import the Keras to DML wrapper and define some basic variables

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:522: FutureWarning: Passing (type, 1) or 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 28, 28)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 14, 14)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 14, 14)        51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 7, 7)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               1606144   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
__________

In [2]:
import math
from systemml.mllearn import Keras2DML
epochs = 5
batch_size = 100
samples = 60000
max_iter = int(epochs*math.ceil(samples/batch_size))


In [3]:
# Now create a SystemML model by calling the Keras2DML method and feeding it your spark session, Keras model, its input shape, and the  # predefined variables. We also ask to be displayed the traning results every 10 iterations.
keras_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
sysml_model = Keras2DML(spark, keras_model, input_shape=(1,28,28), weights='weights_dir', batch_size=batch_size, max_iter=max_iter, test_interval=0, display=10)

Py4JJavaError: An error occurred while calling None.org.apache.sysml.api.dl.Caffe2DML.
: com.google.protobuf.TextFormat$ParseException: 22:10: Couldn't parse integer: For input string: "2.5"
	at com.google.protobuf.TextFormat$Tokenizer.parseException(TextFormat.java:912)
	at com.google.protobuf.TextFormat$Tokenizer.integerParseException(TextFormat.java:933)
	at com.google.protobuf.TextFormat$Tokenizer.consumeUInt32(TextFormat.java:753)
	at com.google.protobuf.TextFormat.mergeField(TextFormat.java:1179)
	at com.google.protobuf.TextFormat.mergeField(TextFormat.java:1156)
	at com.google.protobuf.TextFormat.mergeField(TextFormat.java:1156)
	at com.google.protobuf.TextFormat.merge(TextFormat.java:1062)
	at com.google.protobuf.TextFormat.merge(TextFormat.java:1028)
	at com.google.protobuf.TextFormat.merge(TextFormat.java:998)
	at org.apache.sysml.api.dl.Utils$.readCaffeNet(Utils.scala:148)
	at org.apache.sysml.api.dl.CaffeNetwork.<init>(CaffeNetwork.scala:56)
	at org.apache.sysml.api.dl.Caff

In [2]:
################################### Keras2DML: Parallely training neural network with SystemML####################################### 
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Input, Dense, Conv1D, Conv2D, MaxPooling2D, Dropout,Flatten
from keras import backend as K
from keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
import math

K.set_image_data_format("channels_first")

from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()


# Expect to see a numpy n-dimentional array of (60000, 28, 28)

type(X_train), X_train.shape, type(X_train)


#This time however, we flatten each of our 28 X 28 images to a vector of 1, 784

X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

# expect to see a numpy n-dimentional array of : (60000, 784) for Traning Data shape and (10000, 784) for Test Data shape
type(X_train), X_train.shape, X_test.shape


#We also use sklearn's MinMaxScaler for normalizing

from sklearn.preprocessing import MinMaxScaler
def scaleData(data):
    # normalize features
    scaler = MinMaxScaler(feature_range=(0, 1))
    return scaler.fit_transform(data)

X_train = scaleData(X_train)
X_test = scaleData(X_test)


# We define the same Keras model as earlier

input_shape = (1,28,28) if K.image_data_format() == 'channels_first' else (28,28, 1)
keras_model = Sequential()
keras_model.add(Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=input_shape, padding='same'))
keras_model.add(MaxPooling2D(pool_size=(2, 2)))
keras_model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
keras_model.add(MaxPooling2D(pool_size=(2, 2)))
keras_model.add(Flatten())
keras_model.add(Dense(512, activation='relu'))
keras_model.add(Dropout(0.5))
keras_model.add(Dense(10, activation='softmax'))
keras_model.summary()

keras_model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])
# Import the Keras to DML wrapper and define some basic variables

from systemml.mllearn import Keras2DML
epochs = 5
batch_size = 100
samples = 60000
max_iter = int(epochs*math.ceil(samples/batch_size))

# Now create a SystemML model by calling the Keras2DML method and feeding it your spark session, Keras model, its input shape, and the  # predefined variables. We also ask to be displayed the traning results every 10 iterations.

sysml_model = Keras2DML(spark, keras_model, input_shape=(1,28,28), weights='weights_dir', batch_size=batch_size, max_iter=max_iter, test_interval=0, display=10)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 32, 28, 28)        832       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 32, 14, 14)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 14, 14)        51264     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 64, 7, 7)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               1606144   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
__________

Py4JJavaError: An error occurred while calling None.org.apache.sysml.api.dl.Caffe2DML.
: com.google.protobuf.TextFormat$ParseException: 22:10: Couldn't parse integer: For input string: "2.5"
	at com.google.protobuf.TextFormat$Tokenizer.parseException(TextFormat.java:912)
	at com.google.protobuf.TextFormat$Tokenizer.integerParseException(TextFormat.java:933)
	at com.google.protobuf.TextFormat$Tokenizer.consumeUInt32(TextFormat.java:753)
	at com.google.protobuf.TextFormat.mergeField(TextFormat.java:1179)
	at com.google.protobuf.TextFormat.mergeField(TextFormat.java:1156)
	at com.google.protobuf.TextFormat.mergeField(TextFormat.java:1156)
	at com.google.protobuf.TextFormat.merge(TextFormat.java:1062)
	at com.google.protobuf.TextFormat.merge(TextFormat.java:1028)
	at com.google.protobuf.TextFormat.merge(TextFormat.java:998)
	at org.apache.sysml.api.dl.Utils$.readCaffeNet(Utils.scala:148)
	at org.apache.sysml.api.dl.CaffeNetwork.<init>(CaffeNetwork.scala:56)
	at org.apache.sysml.api.dl.Caff

In [7]:
keras_model.layers